# Kernel Formatting Demonstration

This notebook demonstrates the new kernel formatting capabilities with `str()`. Examples progress from simple to complex.

In [ ]:
import sys

sys.path.insert(0, "..")

import jax.numpy as jnp

from kernax import ConstantKernel, LinearKernel, SEKernel, WhiteNoiseKernel
from kernax.WrapperKernels import ExpKernel, LogKernel
from kernax.engines import SafeDiagonalEngine

## 1. Simple Kernels with Scalar Parameters

Simple kernels display their hyperparameters in a readable format.

In [2]:
# Constant kernel with scalar
k_const = ConstantKernel(value=2.5)
print(f"ConstantKernel: {k_const}")
print()

ConstantKernel: 2.50



In [3]:
# SE (Squared Exponential) kernel
k_se = SEKernel(length_scale=1.0)
print(f"SEKernel: {k_se}")
print()

SEKernel: SEKernel(length_scale=1.00)



In [4]:
# Linear kernel with multiple parameters
k_linear = LinearKernel(variance_b=0.5, variance_v=1.0, offset_c=0.0)
print(f"LinearKernel: {k_linear}")
print()

LinearKernel: LinearKernel(variance_b=0.50, variance_v=1.00, offset_c=0.00)



## 2. Kernels with Multidimensional Parameters

When a parameter is an array, the format displays `[mean ± std]_shape` with subscript indices.

In [5]:
# Constant kernel with 1D array
k_const_array = ConstantKernel(value=jnp.array([1.0, 2.0, 3.0]))
print(f"ConstantKernel (1D array): {k_const_array}")
print()

ConstantKernel (1D array): [2.00 ± 0.82]₃



In [6]:
# Constant kernel with 2D array
k_const_matrix = ConstantKernel(value=jnp.array([[1.0, 2.0], [3.0, 4.0]]))
print(f"ConstantKernel (2D array): {k_const_matrix}")
print()

ConstantKernel (2D array): [2.50 ± 1.12]₂ₓ₂



In [7]:
# SE kernel with vector length_scale (for ARD)
k_se_vector = SEKernel(length_scale=jnp.array([0.5, 1.0, 1.5, 2.0]))
print(f"SEKernel (vector length_scale): {k_se_vector}")
print()

SEKernel (vector length_scale): SEKernel(length_scale=[1.25 ± 0.56]₄)



## 3. Kernel Operators - Addition

The `+` operator creates a `SumKernel` that displays intuitively.

In [8]:
# Addition of two simple kernels
k_sum = SEKernel(length_scale=1.0) + ConstantKernel(value=0.5)
print(f"SE + Constant: {k_sum}")
print()

SE + Constant: SEKernel(length_scale=1.00) + 0.50



In [9]:
# Addition of multiple kernels
k_sum_multi = SEKernel(length_scale=1.0) + SEKernel(length_scale=2.0) + ConstantKernel(value=1.0)
print(f"SE + SE + Constant: {k_sum_multi}")
print()

SE + SE + Constant: SEKernel(length_scale=1.00) + SEKernel(length_scale=2.00) + 1.00



## 4. Kernel Operators - Subtraction

Subtraction uses `NegKernel` internally, but displays as natural subtraction.

In [10]:
# Kernel subtraction
k_diff = SEKernel(length_scale=1.0) - ConstantKernel(value=0.5)
print(f"SE - Constant: {k_diff}")
print()

SE - Constant: SEKernel(length_scale=1.00) - 0.50



In [11]:
# Kernel negation
k_neg = -SEKernel(length_scale=1.0)
print(f"Negation: {k_neg}")
print()

Negation: - SEKernel(length_scale=1.00)



## 5. Kernel Operators - Multiplication

The `*` operator creates a `ProductKernel`.

In [12]:
# Product of two kernels
k_prod = SEKernel(length_scale=1.0) * LinearKernel(variance_b=0.5, variance_v=1.0, offset_c=0.0)
print(f"SE × Linear: {k_prod}")
print()

SE × Linear: SEKernel(length_scale=1.00) * LinearKernel(variance_b=0.50, variance_v=1.00, offset_c=0.00)



In [13]:
# Product with negative kernel (adds parentheses)
k_prod_neg = SEKernel(length_scale=1.0) * (-ConstantKernel(value=2.0))
print(f"SE × (-Constant): {k_prod_neg}")
print()

SE × (-Constant): SEKernel(length_scale=1.00) * (- 2.00)



## 6. Wrapper Kernels

Wrappers apply transformations to kernels and display using functional notation.

In [14]:
# Exp wrapper
k_exp = ExpKernel(SEKernel(length_scale=1.0))
print(f"Exp wrapper: {k_exp}")
print()

Exp wrapper: Exp(SEKernel(length_scale=1.00))



In [15]:
# Log wrapper
k_log = LogKernel(SEKernel(length_scale=1.0))
print(f"Log wrapper: {k_log}")
print()

Log wrapper: Log(SEKernel(length_scale=1.00))



In [ ]:
# WhiteNoise kernel (diagonal matrix with constant value)
k_white_noise = WhiteNoiseKernel(noise=1.0)
print(f"WhiteNoise kernel: {k_white_noise}")
print()

## 7. Simple Combinations

Combinations of operators and wrappers.

In [17]:
# Wrapper on a sum
k_exp_sum = ExpKernel(SEKernel(length_scale=1.0) + ConstantKernel(value=0.5))
print(f"Exp(SE + Constant): {k_exp_sum}")
print()

Exp(SE + Constant): Exp(SEKernel(length_scale=1.00) + 0.50)



In [18]:
# Sum of wrappers
k_sum_wrappers = ExpKernel(SEKernel(length_scale=1.0)) + LogKernel(
	LinearKernel(variance_b=1.0, variance_v=1.0, offset_c=0.0)
)
print(f"Exp(SE) + Log(Linear): {k_sum_wrappers}")
print()

Exp(SE) + Log(Linear): Exp(SEKernel(length_scale=1.00)) + Log(LinearKernel(variance_b=1.00, variance_v=1.00, offset_c=0.00))



In [ ]:
# Product with diagonal computation engine
k_prod_diag = SEKernel(length_scale=1.0) * ConstantKernel(value=2.0, computation_engine=SafeDiagonalEngine)
print(f"SE × Constant (diagonal): {k_prod_diag}")
print()

## 8. Complex Nested Structures

Demonstration of deeply nested kernel structures.

In [20]:
# Multiple wrapping levels
k_nested_wrapper = ExpKernel(LogKernel(SEKernel(length_scale=1.0)))
print(f"Double wrapper: {k_nested_wrapper}")
print()

Double wrapper: Exp(Log(SEKernel(length_scale=1.00)))



In [ ]:
# Complex combination: (Exp(SE) + Log(Linear)) × Constant(diagonal)
k_complex1 = (
	ExpKernel(SEKernel(length_scale=1.0))
	+ LogKernel(LinearKernel(variance_b=1.0, variance_v=1.0, offset_c=0.0))
) * ConstantKernel(value=2.0, computation_engine=SafeDiagonalEngine)
print("Complex combination 1:")
print(f"{k_complex1}")
print()

In [22]:
# Expression with nested subtraction and multiplication
k_complex2 = (SEKernel(length_scale=1.0) + SEKernel(length_scale=2.0)) * (
	LinearKernel(variance_b=1.0, variance_v=1.0, offset_c=0.0) - ConstantKernel(value=0.5)
)
print("Complex combination 2:")
print(f"{k_complex2}")
print()

Complex combination 2:
SEKernel(length_scale=1.00) + SEKernel(length_scale=2.00) * LinearKernel(variance_b=1.00, variance_v=1.00, offset_c=0.00) - 0.50



In [ ]:
# Very nested structure with all operator types
k_very_complex = ExpKernel(
	(SEKernel(length_scale=1.0) * ConstantKernel(value=2.0))
	+ WhiteNoiseKernel(noise=0.5)
	- LogKernel(SEKernel(length_scale=0.5))
)
print("Very complex structure:")
print(f"{k_very_complex}")
print()

## 9. Expressions with Multidimensional Parameters

Complex combinations including vector/matrix parameters.

In [24]:
# Sum of kernels with vector parameters
k_vector_params = SEKernel(length_scale=jnp.array([1.0, 2.0, 3.0])) + ConstantKernel(
	value=jnp.array([0.5, 0.5, 0.5])
)
print("SE (vector) + Constant (vector):")
print(f"{k_vector_params}")
print()

SE (vector) + Constant (vector):
SEKernel(length_scale=[2.00 ± 0.82]₃) + [0.50 ± 0.00]₃



In [25]:
# Wrapper of kernel with matrix parameters
k_matrix_wrapper = ExpKernel(ConstantKernel(value=jnp.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])))
print("Exp(Constant matrix):")
print(f"{k_matrix_wrapper}")
print()

Exp(Constant matrix):
Exp([3.50 ± 1.71]₃ₓ₂)



## 10. Edge Cases and Long Expressions

Demonstration of edge cases to test formatting limits.

In [26]:
# Long chain of additions
k_long_chain = (
	SEKernel(length_scale=1.0)
	+ SEKernel(length_scale=2.0)
	+ SEKernel(length_scale=3.0)
	+ ConstantKernel(value=1.0)
	+ ConstantKernel(value=2.0)
)
print("Long chain:")
print(f"{k_long_chain}")
print()

Long chain:
SEKernel(length_scale=1.00) + SEKernel(length_scale=2.00) + SEKernel(length_scale=3.00) + 1.00 + 2.00



In [27]:
# Expression with multiple levels of parentheses
k_parentheses = (
	(SEKernel(length_scale=1.0) + ConstantKernel(value=1.0))
	* (LinearKernel(variance_b=1.0, variance_v=1.0, offset_c=0.0) - ConstantKernel(value=0.5))
) + ExpKernel(SEKernel(length_scale=0.5))
print("Expression with multiple parentheses:")
print(f"{k_parentheses}")
print()

Expression with multiple parentheses:
SEKernel(length_scale=1.00) + 1.00 * LinearKernel(variance_b=1.00, variance_v=1.00, offset_c=0.00) - 0.50 + Exp(SEKernel(length_scale=0.50))



In [ ]:
# Maximum complexity expression
k_max_complexity = ExpKernel(
	LogKernel(
		(SEKernel(length_scale=jnp.array([1.0, 2.0])) * ConstantKernel(value=3.0, computation_engine=SafeDiagonalEngine))
		+ (
			LinearKernel(variance_b=1.0, variance_v=2.0, offset_c=0.5)
			- ExpKernel(SEKernel(length_scale=0.5))
		)
	)
) * ConstantKernel(value=jnp.array([[1.0, 2.0], [3.0, 4.0]]))

print("Maximum complexity expression:")
print(f"{k_max_complexity}")
print()

## Conclusion

This notebook demonstrated the kernel formatting capabilities:

1. **Simple kernels**: Clear display of hyperparameters
2. **Multidimensional parameters**: `[mean ± std]_shape` format with subscript indices
3. **Operators**: Intuitive notation for `+`, `-`, and `*`
4. **Wrappers**: Functional notation (Exp, Log, Diag, etc.)
5. **Combinations**: Correct handling of priorities and parentheses
6. **Nesting**: Support for arbitrarily complex structures

The formatting system makes kernel expressions readable while preserving their mathematical structure.